In [ ]:
# default_exp graph

In [ ]:
#hide
%load_ext autoreload
%autoreload 2

# Graph module

> Provides main class and helpers classes and functions to handle pangenome graph and related data structures

In [ ]:
#hide
from nbdev.showdoc import *
from nbdev.export import notebook2script
notebook2script()

Converted 00_init.ipynb.
Converted 01_graph.ipynb.
Converted 02_tree.ipynb.
Converted 03_synteny.ipynb.
Converted 04_utils.ipynb.
Converted index.ipynb.


In [ ]:
#exporti

import os
import glob
import json
import itertools
import warnings

import networkx as nx

import pandas as pd
import numpy as np

from skbio.io import read as skbio_read
from skbio.metadata import IntervalMetadata
from skbio.sequence import DNA

from dnasim.IO import writeFASTA
from dnasim.simulation import inverseSequence

from pangraph_constructor.synteny import processAccession,generatePathsLinks
from pangraph_constructor.utils import bidict
from pangraph_constructor.tree import TremauxTree

In [ ]:
#export
class LinkGetter:
    def __init__(self,nodes,links):
        self.links = links
        self.nodes = nodes
    
    def __getitem__(self,nodeID):
        try:
            node = self.links[nodeID]
            childrenList = list(itertools.chain(*list(node.values())))
            return list(set([el[0] for el in childrenList]))
        except KeyError:
            if nodeID>0 and nodeID<=len(self.nodes):
                return []
            else:
                raise KeyError(f'There is no node with ID {nodeID}')
    def __call__(self):
        allLinks = []
        for node in range(1,len(self.nodes)+1):
            res = self[node]
            if res is not None:
                allLinks.extend([(node,el) for el in res])
        return allLinks
#         return list(self)
    def __iter__(self):
        for i in range(len(self.nodes)):
            res = self[i+1]
            if res is not None:
                yield (i+1,res)
            else:
                yield (i+1,[])

In [ ]:
#export
class GenomeGraph:
    def __init__(self,gfaPath=None,
                 paths=None,
                 nodes=None,nodesData=None,links=None,
                 sequenceFiles=None,annotationFiles=None,
                 doBack=False,**kwargs):
        self.nodes = []
        self.nodesData = []
        self.forwardLinks = {}
        self.paths = []
        self.accessions = []
        self._nxGraph = None
        self.tremauxTree = None
        
        self.isBack = doBack
        
        if gfaPath is not None:
            self._loadGFA(gfaPath,isGFASeq=kwargs.get('isGFASeq',True))
        elif nodes is not None and links is not None and paths is not None:
            self.nodes = nodes
            if nodesData is not None:
                self.nodesData = nodesData
            else:
                self.nodesData = ['']*len(self.nodes)
            self.forwardLinks = links
            self.paths = paths
        elif paths is not None and sequenceFiles is not None:
            self.nodes,self.nodesData,self.forwardLinks = self.graphFromPaths(paths,sequenceFiles) # sequenceFiles can be None
            self.paths = paths
        elif annotationFiles is not None:
            self._graphFromAnnotation(annotationFiles,sequenceFiles,**kwargs)
        
        self.order = list(range(1,len(self.nodes)+1))
        
        self.pureForwardlinks = LinkGetter(self.nodes,self.forwardLinks)
        
        self._pathCount()
        
        if doBack:
            self.backLinks = self._revertLinks(self.forwardLinks)
            self.pureBackLinks = LinkGetter(self.nodes,self.backLinks)
    
    # check edgePaths calculation. Incorrectly calculates the numbers 
    # (one too many in one of group of incoming edges to each node)
    def _pathCount(self):
        # self.inPath = []
        # self.outPath = []
        self.nodePass = [0]*len(self.nodes)
        self.nodeStrandPaths = [[0,0] for _ in range(len(self.nodes))]
        self.pathStarts = [0]*len(self.nodes)
        self.inPath = [0]*len(self.nodes)
        self.outPath = [0]*len(self.nodes)
        self.edgePaths = {}
        
        for path in self.paths:
            
            previousNode = None
            for nodeStrand in path:
                pathNode = self.nodeNameToID[nodeStrand[:-1]]-1
                pathStrand = int(nodeStrand[-1]=='-') # 0 if "+", 1 if "-"
                self.nodePass[pathNode] += 1
                self.nodeStrandPaths[pathNode][pathStrand] += 1
                self.outPath[pathNode] += 1
                self.inPath[pathNode] += 1
                if previousNode is not None:
                    self.edgePaths.setdefault((previousNode+1,pathNode+1),0)
                    self.edgePaths[(previousNode+1,pathNode+1)] += 1
                else:
                    self.pathStarts[pathNode] += 1
                    self.inPath[pathNode] -= 1
                previousNode = pathNode
            self.outPath[previousNode] -= 1
    
    def _revertLinks(self,forwardLinks):
        backLinks = {}
        for fromNome,links in forwardLinks.items():
            for fromStrand,strandLinks in links.items():
                for item in strandLinks:
                    if item[0] not in backLinks.keys():
                        backLinks[item[0]] = {}
                        backLinks[item[0]][item[1]] = []

                    elif item[1] not in forwardLinks[item[0]].keys():
                        backLinks[item[0]][item[1]] = []

                    backLinks[item[0]][item[1]].append([fromNode,fromStrand])
        return backLinks
    
    def _loadGFA(self,gfaFile,isGFASeq=True):
        baseName = os.path.splitext(os.path.basename(gfaFile))[0]
        dirPath = os.path.dirname(gfaFile)
        jsonFile = f'{dirPath}{os.path.sep}{baseName}.json'
        
        if os.path.exists(jsonFile):
            nodeNames = json.load(open(jsonFile,mode='r'))
        else:
            nodeNames = None
        
        prevPathSegment = None
        
        gfaList = open(gfaFile,mode='r').readlines()
        
        if gfaList[0][7:-1] == '2.0':
            raise NotImplementedError('At the moment only GFA v1 is supported')
        elif gfaList[0][7:-1] != '1.0':
            warnings.warn('Cannot identify version of GFA. Assuming it is GFA v1. If it is not, expect unpredictable results.',category=ResourceWarning)
       
        segmentList = [el for el in gfaList if el.lower().startswith('s')]
        linkList = [el for el in gfaList if el.lower().startswith('l')]
        pathStringsList = [el for el in gfaList if el.lower().startswith('p')]

        self.nodeNameToID = {}
        
        for nodeID,node in enumerate(segmentList):
            _,segID,segGFAData = node.rstrip().split(sep='\t')
            if isGFASeq:
                segSeq = segGFAData
            else:
                segSeq = ''
            
            if nodeNames is not None:
                segName = nodeNames[int(segID)-1]
            elif isGFASeq:
                segName = str(segID)
            else:
                segName = segGFAData
            
            self.nodeNameToID[segID] = len(self.nodes)+1
            
            self.nodes.append(segName)
            self.nodesData.append(segSeq)
            
        for link in linkList:
            _,fromNodeID,fromStrand,toNodeID,toStrand,_ = link.rstrip().split(sep='\t')
            fromNode = self.nodeNameToID[fromNodeID]
            toNode = self.nodeNameToID[toNodeID]
            
            curLink = self.forwardLinks.setdefault(fromNode,{})
            curStrand = curLink.setdefault(fromStrand,[])
            curStrand.append((toNode,toStrand))
        
        for pathString in pathStringsList:
            _,seqID,path,_ = pathString.rstrip().split(sep='\t')
            self.paths.append(path.split(','))
            self.accessions.append(seqID)
                 
    def _graphFromNodesLinks(self,nodes,links):
        raise NotImplementedError('Generating graph from nodes and links is not yet implemented.')
    
    def _graphFromPaths(self,paths,sequenceFiles=None):
        raise NotImplementedError('Generating graphs from paths is not implemented yet.')
    
    def _graphFromAnnotation(self,annotationFiles,sequenceFiles=None,**kwargs):
        
        try:
            fileOrder = kwargs['fileOrder']
        except KeyError:
            fileOrder = list(range(len(annotationFiles)))
            
        doUS = kwargs['doUS']
        self.usCounter = 0
        self.OGList = []
        links = self._linksDictToSet(self.forwardLinks)
        
        for fileNum in fileOrder:
            if sequenceFiles is not None:
                seqFile = sequenceFiles[fileNum]
            else:
                seqFile = None
            accessionID,genes,sequences = processAccession(annotationFiles[fileNum],seqFile)
            try:
                seqList = [f'{accessionID}{kwargs["seqSuffix"]}']
            except KeyError:
                seqList = genes.sequenceID.unique().tolist()
                seqList.sort()
            path = []
            for seqID in seqList:
                
                p,cigar,usCounter = generatePathsLinks(genes.loc[genes.sequenceID==seqID],sequences,self.OGList,
                                                       self.nodes,links,
                                                       self.usCounter,doUS=doUS,segmentData=self.nodesData)
                path = path + p
            self.paths.append(path)
            self.accessions.append(accessionID)
            
        
        self.forwardLinks = self._linksSetToDict(links)
    
    def _linksDictToSet(self,dictLinks):
        setLinks = {}
        for fromNode,strandLinks in dictLinks.items():
            for fromStrand,linkList in strandLinks.items():
                for toNode,toStrand in linkList:
                    fromNodeStrand = f'{fromNode}{fromStrand}'
                    toNodeStrand = f'{toNode}{toStrand}'
                    if fromNodeStrand not in setLinks.keys():
                        setLinks[fromNodeStrand] = set()
                    setLinks[fromNodeStrand].add(toNodeStrand)
        return setLinks
    
    def _linksSetToDict(self,setLinks):
        dictLinks = {}
        for fromNodeStrand,links in setLinks.items():
            fromNode = int(fromNodeStrand[:-2])
            fromStrand = fromNodeStrand[-1]
            for toNodeStrand in links:
                toNode = int(toNodeStrand[:-2])
                toStrand = toNodeStrand[-1]
                
                if fromNode not in dictLinks.keys():
                    dictLinks[fromNode] = {}
                    dictLinks[fromNode][fromStrand] = []

                elif fromStrand not in dictLinks[fromNode].keys():
                    dictLinks[fromNode][fromStrand] = []

                dictLinks[fromNode][fromStrand].append((toNode,toStrand))
        return dictLinks
    
    def generateTremauxTree(self,byPath=True):
        _nxGraph = nx.DiGraph()
        
        for nodeID,toList in self.pureForwardlinks:
            if toList is not None:
                edgeList = [(nodeID,toItem) for toItem in toList]
                _nxGraph.add_edges_from(edgeList)
            
        self.tremauxTree = TremauxTree(_nxGraph,self,byPath)
        
    def addAccessionAnnotation(self,annotationFile,sequenceFile=None):
        '''
        Ideally, a function should be able to add one accesstion to existing graph. 
        When implemented, `_graphFromAnnotation` should be using this function.
        
        '''
        pass
    
    def addLink(self,fromNode,fromStrand,toNode,toStrand):
        # Need testing
        if fromNode not in self.nodes:
            raise IndexError(f'Node {fromNode} is not in nodes list')
        if toNode not in self.nodes:
            raise IndexError(f'Node {toNode} is not in nodes list')
        
        if fromNode not in self.forwardLinks.keys():
            self.forwardLinks[fromNode] = {}
            self.forwardLinks[fromNode][fromStrand] = []
            
        elif fromStrand not in self.forwardLinks[fromNode].keys():
            self.forwardLinks[fromNode][fromStrand] = []

        self.forwardLinks[fromNode][fromStrand].append([toNode,toStrand])

        if self.isBack:
            if toNode not in self.backLinks.keys():
                self.backLinks[toNode] = {}
                self.backLinks[toNode][toStrand] = []

            elif toStrand not in self.backLinks[toNode].keys():
                self.backLinks[toNode][toStrand] = []

            self.backLinks[toNode][toStrand].append([fromNode,fromStrand])
    
    def addNode(self,nodeID,data=None):
        # Need testing
        if nodeID not in self.nodes:
            self.nodes.append(nodeID)
            self.nodesData[nodeID] = data
        else:
            warnings.warn(f'You attempted to add node with {nodeID}, but it already exists in the graph. The addition was ignored.')
        pass
    
    def _getStartNodes(self,bubbleNode):
        allPaths = list(nx.shortest_path(self.tremauxTree,None,bubbleNode).values())
        startList = []
        for path in allPaths:
            if len(startList)>0:
                for node in startList.copy():
                    if nx.has_path(self.tremauxTree,node,path[0]):
                        continue
                    if nx.has_path(self.tremauxTree,path[0],node):
                        startList.remove(node)

            startList.append(path[0])

        return startList
    
    def treeSort(self,byPath=True,bubblePriorityThreshold=0.5):
        
        print('Constructing Tremaux tree')
        self.generateTremauxTree(byPath)
        print('Done!')
        queue = []
        processed = []
        self.order = []
        
        print('Getting root nodes')
        rootNodes = self.tremauxTree.getRootNodes()[::-1]
#         self.order.append(rootNode)
        startNode = rootNodes.pop()
        self.order.append(startNode)
        descendantsToAdd = list(self.tremauxTree.edges(startNode))
        descendantsToAdd.sort(key=lambda edge: self.edgePaths.get(edge,0))
        queue.extend(descendantsToAdd)
        
        print('Start Loop...')
        while len(queue)>0 or len(rootNodes)>0:
#             pdb.set_trace()
            if len(queue)==0:
                try:
                    while len(queue)==0:
                        startNode = rootNodes.pop()
                        descendantsToAdd = list(self.tremauxTree.edges(startNode))
                        if len(descendantsToAdd)>0:
                            descendantsToAdd.sort(key=lambda edge: self.edgePaths.get(edge,0))
                            queue.extend([edge for edge in descendantsToAdd if edge not in processed])
                        if startNode not in self.order:
                            self.order.append(startNode)
                except IndexError:
                    break
            
            startNode,endNode = queue.pop()
            
            print(f'{len(queue)} - {len(processed)} - {len(self.order)} - {startNode} - {endNode}')
            
            if (startNode,endNode) in processed:
                continue
            if (startNode in self.order and endNode in self.order):
                if not ((startNode,endNode) in processed):
                    processed.append((startNode,endNode))
                continue
            
            endNodeAdded = False
            
            bubblePriorityQueue = []
            bubbleEdges = self.tremauxTree.bubbleEdges.inverse.get(endNode,[])
            if len(bubbleEdges)>0:
                if byPath:
                    bubbleEdgesOrder = np.argsort([self.edgePaths.get((edge,endNode),0) for edge in bubbleEdges])[::-1]
                else:
                    bubbleEdgesOrder = list(range(len(bubbleEdges)))
                # Chech that bubble node is not in order already
                # Check edgePaths for all outgoing links (including non-bubbles but not loops), use this bubble 
                # only if the bubble edge has maximum number of paths.
                # if this bubble should be used compare edgePaths for bubble edge and for normal edge to 
                # current node. If normal edge is larger, then add current node to order and node past LCA 
                # on alternative path(s) to the queue (to be processed next) and after them the descendandts of 
                # current node. Otherwise, do not add current node to order, but add nodes past LCA and then (to processed later) current node to queue.
                
                for bEdge in bubbleEdgesOrder:
                    bubbleNode = bubbleEdges[bEdge]
                    if bubbleNode in self.order:
                        continue
                    if byPath:
                        curBubbleEdgePaths = self.edgePaths.get((bubbleNode,endNode),0)
                        bubbleNodeOutEdges = list(self.tremauxTree.out_edges(bubbleNode)) + \
                                            [(bubbleNode,node) for node in self.tremauxTree.bubbleEdges[bubbleNode]]
                        
                        bubbleNodeEdgePathsMax = max([self.edgePaths.get(edge,0) for edge in bubbleNodeOutEdges])
                        if bubbleNodeEdgePathsMax>curBubbleEdgePaths:
                            continue
                    
                        if self.edgePaths[(bubbleNode,endNode)]/self.outPath[bubbleNode-1]>=bubblePriorityThreshold:
                            bubblePriority = True
                        else:
                            bubblePriority = False
                        
                    lca = nx.lowest_common_ancestor(self.tremauxTree,endNode,bubbleNode)
                    if byPath:
                        if lca is None:
                            startBubbleNodes = self._getStartNodes(bubbleNode)
                            startBubbleNodes.sort(key=lambda node: self.nodePass[node-1])
                            lcaDescendantsToProcess = []
                            for startBubbleNode in startBubbleNodes: 
                                snDescendantsToProcess = [(startBubbleNode,node) for node in list(self.tremauxTree[startBubbleNode])\
                                                          if node not in self.order and nx.has_path(self.tremauxTree,node,bubbleNode)]
                                snDescendantsToProcess.sort(key=lambda node: self.edgePaths.get((startBubbleNode,node),0))
                                lcaDescendantsToProcess.extend(snDescendantsToProcess)
                        else: 
                            lcaDescendantsToProcess = [(lca,node) for node in list(self.tremauxTree[lca]) \
                                                       if node not in self.order and nx.has_path(self.tremauxTree,node,bubbleNode)]
                            lcaDescendantsToProcess.sort(key=lambda node: self.edgePaths.get((lca,node),0))
                        
                        for edge in lcaDescendantsToProcess:
                            if edge in queue:
                                queue.remove(edge)
                        if len(lcaDescendantsToProcess)>0:
                            if bubblePriority:
                                bubblePriorityQueue.append((bubbleNode,[edge for edge in lcaDescendantsToProcess if edge not in processed]))
                            else:
                                queue.extend([edge for edge in lcaDescendantsToProcess if edge not in processed])
                    else:
                        # TODO!!!
                        # rewrite pure graph sorting to process alternative branches by adding the top nodes 
                        # of each branch to the queue for immediate processing.
                        pathToAdd = nx.shortest_path(self.tremauxTree,lca,bubbleNode)
                        if lca is None:
                            allPaths = list(pathToAdd.values())
                            allPaths.sort(key=len)
                            pathToAdd = allPaths[-1]
                        else:
                            if endNode not in self.order and byPath:
                                self.order.append(endNode)
                                endNodeAdded = True
                        # WRONG!
                        self.order.extend([node for node in pathToAdd if node not in self.order])

            
            
            if len(bubblePriorityQueue)>0:
                queue.append((startNode,endNode))#Why would I add the processed block again? Should I add descendant edges of the end node here? 
                # It should save a lot of wasted time and speed up the process.
                bubblePriorityQueue.sort(key=lambda b: self.edgePaths[b[0],endNode])
                while len(bubblePriorityQueue)>0:
                    bNode,bToAdd = bubblePriorityQueue.pop()
                    queue.extend(bToAdd)
            else:
                processed.append((startNode,endNode))
                if endNode not in self.order:
                    self.order.append(endNode)
                    endNodeAdded = True
            
                edgesToAdd = [edge for edge in list(self.tremauxTree.edges(endNode)) if edge not in processed and edge not in queue]
                edgesToAdd.sort(key=lambda edge: self.edgePaths.get(edge,0))
                queue.extend(edgesToAdd)
             
    def _generateIsInversionDict(self):
#         setLinks = self._linksDictToSet(self.forwardLinks)
#         uniqueNodeStrands = list(set(list(setLinks.keys())).union(*list(setLinks.values())))
        
#         uniqueNodeStrands.sort(key=lambda val: int(val[:-1])) # Is it really necessary if we build a dict
        
#         nodeStrandArray = np.array([(v[:-1],v[-1]) for v in uniqueNodeStrands])
#         numStrandsPerNode = np.unique(nodeStrandArray[:,0],return_counts=True)
        
#         isInversionDict = dict([(int(node),numStrands<=1) for node,numStrands in zip(*numStrandsPerNode)])
        
        isInversionDict = dict([(int(node+1),(self.nodeStrandPaths[node][1]>self.nodeStrandPaths[node][0])) for node in range(len(self.nodes))])
    
        return isInversionDict
    
    def toGFA(self,gfaFile,doSeq=True):
        baseName = os.path.splitext(os.path.basename(gfaFile))[0]
        dirPath = os.path.dirname(gfaFile)
        jsonFile = f'{dirPath}{os.path.sep}nodeNames_{baseName}.json'
        
        gfaWriter = open(gfaFile,mode='w')
        
        gfaWriter.write('H\tVN:Z:1.0\n')
        
        translator = {}
        nodeNameList = []
        
        for i,nodeID in enumerate(self.order):
            nodeSeq = self.nodesData[nodeID-1]
            nodeName = self.nodes[nodeID-1]
            if doSeq and len(nodeSeq)>0:
                gfaWriter.write(f'S\t{i+1}\t{nodeSeq}\n')
                nodeNameList.append(nodeName)
            else:
                gfaWriter.write(f'S\t{i+1}\t{nodeName}\n')
            
            translator[nodeID] = i+1
        
        isInversionDict = self._generateIsInversionDict()
        strandInversionDict = {'+':'-','-':'+'}
        
        for fromNode,strandLinks in self.forwardLinks.items():
            for fromStrand,toLinkStrands in strandLinks.items():
                for toNode,toStrand in toLinkStrands:
                    
                    if isInversionDict.get(fromNode,False):
                        _fromStrand = strandInversionDict[fromStrand]
                    else:
                        _fromStrand = fromStrand

                    if isInversionDict.get(toNode,False):
                        _toStrand = strandInversionDict[toStrand]
                    else:
                        _toStrand = toStrand
                    gfaWriter.write(f'L\t{translator[fromNode]}\t{_fromStrand}\t{translator[toNode]}\t{_toStrand}\t*\n')
        
        for path,accessionID in zip(self.paths,self.accessions):
            newPath = []
            for nodeStrand in path:
                node = self.nodeNameToID[nodeStrand[:-1]]
                if isInversionDict.get(node,False):
                    strand = strandInversionDict[nodeStrand[-1]]
                else:
                    strand = nodeStrand[-1]
                newPath.append(f'{translator[node]}{strand}')
            gfaWriter.write(f'P\t{accessionID}\t{",".join(newPath)}\t*\n')
                
        gfaWriter.write('\n')
        gfaWriter.close()
        
        if len(nodeNameList)==len(self.nodes):
            with open(jsonFile,'w') as jsf:
                json.dump(nodeNameList,jsf)
        
    def __iter__(self,forward=True):
        print(forward)
        for i in range(len(self.nodes)):
            yield i+1,self.forwardLinks[i+1]
    
    def __getitem__(self,ind):
        if isinstance(ind,tuple):
            nodeID = ind[0]
            forward = ind[1]
        else:
            nodeID = ind
            forward = True
            
        if nodeID>len(self.nodes):
            raise IndexError('The nodeID index {nodeID} is out of range')
            
        if not forward and self.isBack:
            try:
                return self.backLinks[nodeID]
            except KeyError:
                return None
        else:
            try:
                return self.forwardLinks[nodeID]
            except KeyError:
                return None

In [ ]:
notebook2script()

Converted 00_init.ipynb.
Converted 01_graph.ipynb.
Converted 02_tree.ipynb.
Converted 03_synteny.ipynb.
Converted 04_utils.ipynb.
Converted index.ipynb.
